# ...

# Libraries and Functions

In [15]:
# libraries imported and used throught the code
import pandas as pd
import sklearn as sklearn
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
import os
import ipykernel
import pathlib

In [16]:
# functions created and used throught the code

def col_unique_5(df):
    ' Show columns wiht under 10 unique values'

    for column in df.columns:
        unique_values = df[column].nunique()
        if unique_values <= 5:
            print(f'{column}: {df[column].unique()}')

def drop_na_cols(df1, df2, threshold=80):
    ''' Drop columns with threshold% or more missing values in both datasets '''

    # calculate percentage of missing values
    missing_values_train = (df1.isnull().sum() / len(df1)) * 100
    missing_values_test = (df2.isnull().sum() / len(df2)) * 100

    # identify columns in both datasets
    columns_to_drop_train = missing_values_train[missing_values_train >= threshold].index
    columns_to_drop_test = missing_values_test[missing_values_test >= threshold].index

    # get the common columns to drop
    common_columns_to_drop = columns_to_drop_train.intersection(columns_to_drop_test)

    # drop columns
    df1 = df1.drop(columns=common_columns_to_drop)
    df2 = df2.drop(columns=common_columns_to_drop)

    return df1, df2

def reduce_mem(df):
    '''Reduces the memory usage of a dataframe'''

    # original memory usage
    print(f'Original memory usage:\n')
    df.info(memory_usage='deep', verbose=False)

    # dictionary with the optimized data types
    dtypes = {'int': [np.int8, np.int16, np.int32, np.int64],
        'float': [np.float16, np.float32, np.float64]}

    for col in df.columns:
        col_type = df[col].dtype

        # type numeric
        if col_type.kind in ['i', 'f']:
            # min and max values
            c_min = df[col].min()
            c_max = df[col].max()

            # find the smallest possible data type
            best_dtype = None
            for dtype_name, dtype_list in dtypes.items():
                for dtype in dtype_list:
                    if np.iinfo(dtype if dtype_name == 'int' else np.finfo(dtype)).min <= c_min and \
                       c_max <= np.iinfo(dtype if dtype_name == 'int' else np.finfo(dtype)).max:
                        best_dtype = dtype
                        break
                if best_dtype:
                    break

            # Convert to that dtype, keeping non-finite values as-is
            if best_dtype:
                if df[col].isnull().any() and np.issubdtype(best_dtype, np.integer):
                    # fill NaN values with the median of the column
                    df[col].fillna(df[col].median(), inplace=True)
                df[col] = df[col].astype(best_dtype)

    # new memmory usage
    print(f'\nNew memory usage:\n')
    df.info(memory_usage='deep', verbose=False)

    return df

def lable_enc(df):
    '''Label Encoding function'''
    # create object
    le = LabelEncoder()

    # select columns with object data type
    cols = df.select_dtypes(include=['object']).columns

    # fit and transform
    for col in cols:
        df[col] = le.fit_transform(df[col].astype(str))

    return df

def minmax_scale(df):
    ''' MinMax Scaling function'''

    # create object
    scaler = MinMaxScaler()

    # fit and transform
    df_scaled = scaler.fit_transform(df)

    return df_scaled

def create_features(df):
    ''' Feature Engineering function'''

    for feature in ['TransactionAmt', 'D15']:
        for group in ['card1', 'card4', 'addr1', 'addr2']:
            # new feature that is the ratio of the original feature to its mean grouped by the current group
            df[f'{feature}_to_mean_{group}'] = df[feature] / df.groupby([group])[feature].transform('mean')
            # new feature that is the ratio of the original feature to its standard deviation grouped by the current group
            df[f'{feature}_to_std_{group}'] = df[feature] / df.groupby([group])[feature].transform('std')

    return df

# Loading the Data

The data is broken into two files identity and transaction, which are joined by `TransactionID`. Not all transactions have corresponding identity information.

I will delete the column `isFraud` since I intend to run a unsupervised ML model for my project.

In [17]:
# get the current working directory
cwd = os.getcwd()

# relative paths
rel_path_train_tran = 'raw_data/train_transaction.csv'
rel_path_train_id = 'raw_data/train_identity.csv'
rel_path_test_tran = 'raw_data/test_transaction.csv'
rel_path_test_id = 'raw_data/test_identity.csv'

# absolute paths
abs_path_train_tran = os.path.join(cwd, rel_path_train_tran)
abs_path_train_id = os.path.join(cwd, rel_path_train_id)
abs_path_test_tran = os.path.join(cwd, rel_path_test_tran)
abs_path_test_id = os.path.join(cwd, rel_path_test_id)

# read files
train_tran = pd.read_csv(abs_path_train_tran)
train_id = pd.read_csv(abs_path_train_id)
test_tran = pd.read_csv(abs_path_test_tran)
test_id = pd.read_csv(abs_path_test_id)

# delete isFraud
train_tran = train_tran.drop(columns=['isFraud'])

# Data Exploration

__Categorical Features - Transaction__<br>
`ProductCD`<br>
`card1 - card6`<br>
`addr1, addr2`<br>
`P_emaildomain`<br>
`R_emaildomain`<br>
`M1 - M9`

__Categorical Features - Identity__<br>
`DeviceType`<br>
`DeviceInfo`<br>
`id_12 - id_38`<br><br>
The `TransactionDT` feature is a timedelta from a given reference datetime (not an actual timestamp).

***
Finding the size of our train and test datasets:

In [18]:
print(f'Train transactions: {train_tran.shape}')
print(f'Train identity: {train_id.shape}')

print(f'\nTTest transactions: {test_tran.shape}')
print(f'Test identity: {test_id.shape}')

Train transactions: (590540, 393)
Train identity: (144233, 41)

TTest transactions: (506691, 393)
Test identity: (141907, 41)


***
Overview of the datasets:

In [19]:
# display all columns and rows
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

__Train transactions dataset:__

In [20]:
train_tran.head(10)

,TransactionID,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,D13,D14,D15,M1,M2,M3,M4,M5,M6,M7,M8,M9,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30,V31,V32,V33,V34,V35,V36,V37,V38,V39,V40,V41,V42,V43,V44,V45,V46,V47,V48,V49,V50,V51,V52,V53,V54,V55,V56,V57,V58,V59,V60,V61,V62,V63,V64,V65,V66,V67,V68,V69,V70,V71,V72,V73,V74,V75,V76,V77,V78,V79,V80,V81,V82,V83,V84,V85,V86,V87,V88,V89,V90,V91,V92,V93,V94,V95,V96,V97,V98,V99,V100,V101,V102,V103,V104,V105,V106,V107,V108,V109,V110,V111,V112,V113,V114,V115,V116,V117,V118,V119,V120,V121,V122,V123,V124,V125,V126,V127,V128,V129,V130,V131,V132,V133,V134,V135,V136,V137,V138,V139,V140,V141,V142,V143,V144,V145,V146,V147,V148,V149,V150,V151,V152,V153,V154,V155,V156,V157,V158,V159,V160,V161,V162,V163,V164,V165,V166,V167,V168,V169,V170,V171,V172,V173,V174,V175,V176,V177,V178,V179,V180,V181,V182,V183,V184,V185,V186,V187,V188,V189,V190,V191,V192,V193,V194,V195,V196,V197,V198,V199,V200,V201,V202,V203,V204,V205,V206,V207,V208,V209,V210,V211,V212,V213,V214,V215,V216,V217,V218,V219,V220,V221,V222,V223,V224,V225,V226,V227,V228,V229,V230,V231,V232,V233,V234,V235,V236,V237,V238,V239,V240,V241,V242,V243,V244,V245,V246,V247,V248,V249,V250,V251,V252,V253,V254,V255,V256,V257,V258,V259,V260,V261,V262,V263,V264,V265,V266,V267,V268,V269,V270,V271,V272,V273,V274,V275,V276,V277,V278,V279,V280,V281,V282,V283,V284,V285,V286,V287,V288,V289,V290,V291,V292,V293,V294,V295,V296,V297,V298,V299,V300,V301,V302,V303,V304,V305,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,V322,V323,V324,V325,V326,V327,V328,V329,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,2987000,86400,68.5,W,13926,NaN,150.0,discover,142.0,credit,315.0,87.0,19.0,NaN,NaN,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,14.0,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,13.0,13.0,NaN,NaN,NaN,0.0,T,T,T,M2,F,T,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,117.0,0.0,0.0,0.0,0.0,0.0,117.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,117.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,117.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,credit,325.0,87.0,NaN,NaN,gmail.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,M0,T,T,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [21]:
train_tran.info(memory_usage='deep', verbose=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 590540 entries, 0 to 590539
Columns: 393 entries, TransactionID to V339
dtypes: float64(376), int64(3), object(14)
memory usage: 2.0 GB


In [22]:
col_unique_5(train_tran)

ProductCD: ['W' 'H' 'C' 'S' 'R']
card4: ['discover' 'mastercard' 'visa' 'american express' nan]
card6: ['credit' 'debit' nan 'debit or credit' 'charge card']
M1: ['T' nan 'F']
M2: ['T' nan 'F']
M3: ['T' nan 'F']
M4: ['M2' 'M0' nan 'M1']
M5: ['F' 'T' nan]
M6: ['T' 'F' nan]
M7: [nan 'F' 'T']
M8: [nan 'F' 'T']
M9: [nan 'F' 'T']
V1: [ 1. nan  0.]
V10: [ 0. nan  1.  2.  3.  4.]
V12: [ 1.  0. nan  2.  3.]
V14: [ 1. nan  0.]
V27: [ 0. nan  1.  2.  4.]
V28: [ 0. nan  1.  2.  4.]
V35: [nan  0.  1.  2.  3.]
V41: [nan  1.  0.]
V65: [ 1. nan  0.]
V68: [ 0. nan  1.  2.]
V75: [ 1.  0. nan  2.  3.  4.]
V88: [ 1. nan  0.]
V89: [ 0. nan  1.  2.]
V94: [ 0. nan  1.  2.]
V107: [ 1.  0. nan]
V117: [ 1.  2.  3.  0. nan]
V118: [ 1.  2.  3.  0. nan]
V119: [ 1.  2.  3.  0. nan]
V120: [ 1.  2.  3.  0. nan]
V121: [ 1.  2.  3.  0. nan]
V122: [ 1.  2.  3.  0. nan]


V241: [nan  1.  2.  4.  5.  0.]
V305: [ 1.  2. nan]


__Train identity dataset:__

In [23]:
train_id.head(10)

,TransactionID,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,id_11,id_12,id_13,id_14,id_15,id_16,id_17,id_18,id_19,id_20,id_21,id_22,id_23,id_24,id_25,id_26,id_27,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987004,0.0,70787.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0,NotFound,NaN,-480.0,New,NotFound,166.0,NaN,542.0,144.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,New,NotFound,Android 7.0,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M
1,2987008,-5.0,98945.0,NaN,NaN,0.0,-5.0,NaN,NaN,NaN,NaN,100.0,NotFound,49.0,-300.0,New,NotFound,166.0,NaN,621.0,500.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,New,NotFound,iOS 11.1.2,mobile safari 11.0,32.0,1334x750,match_status:1,T,F,F,T,mobile,iOS Device
2,2987010,-5.0,191631.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,100.0,NotFound,52.0,NaN,Found,Found,121.0,NaN,410.0,142.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Found,Found,NaN,chrome 62.0,NaN,NaN,NaN,F,F,T,T,desktop,Windows
3,2987011,-5.0,221832.0,NaN,NaN,0.0,-6.0,NaN,NaN,NaN,NaN,100.0,NotFound,52.0,NaN,New,NotFound,225.0,NaN,176.0,507.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,New,NotFound,NaN,chrome 62.0,NaN,NaN,NaN,F,F,T,T,desktop,NaN
4,2987016,0.0,7460.0,0.0,0.0,1.0,0.0,NaN,NaN,0.0,0.0,100.0,NotFound,NaN,-300.0,Found,Found,166.0,15.0,529.0,575.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Found,Found,Mac OS X 10_11_6,chrome 62.0,24.0,1280x800,match_status:2,T,F,T,T,desktop,MacOS
5,2987017,-5.0,61141.0,3.0,0.0,3.0,0.0,NaN,NaN,3.0,0.0,100.0,NotFound,52.0,-300.0,Found,Found,166.0,18.0,529.0,600.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Found,Found,Windows 10,chrome 62.0,24.0,1366x768,match_status:2,T,F,T,T,desktop,Windows
6,2987022,-15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NotFound,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2987038,0.0,31964.0,0.0,0.0,0.0,-10.0,NaN,NaN,0.0,0.0,100.0,Found,NaN,-300.0,Found,Found,166.0,15.0,352.0,533.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Found,Found,Android,chrome 62.0,32.0,1920x1080,match_status:2,T,F,T,T,mobile,NaN
8,2987040,-10.0,116098.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,100.0,NotFound,52.0,NaN,Found,Found,121.0,NaN,410.0,142.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Found,Found,NaN,chrome 62.0,NaN,NaN,NaN,F,F,T,T,desktop,Windows
9,2987048,-5.0,257037.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,100.0,NotFound,52.0,NaN,New,NotFound,225.0,NaN,484.0,507.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,New,NotFound,NaN,chrome 62.0,NaN,NaN,NaN,F,F,T,T,desktop,Windows


In [24]:
train_id.info(memory_usage='deep', verbose=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144233 entries, 0 to 144232
Columns: 41 entries, TransactionID to DeviceInfo
dtypes: float64(23), int64(1), object(17)
memory usage: 157.6 MB


In [25]:
col_unique_5(train_id)

id_12: ['NotFound' 'Found']
id_15: ['New' 'Found' nan 'Unknown']
id_16: ['NotFound' 'Found' nan]
id_23: [nan 'IP_PROXY:TRANSPARENT' 'IP_PROXY:ANONYMOUS' 'IP_PROXY:HIDDEN']
id_27: [nan 'Found' 'NotFound']
id_28: ['New' 'Found' nan]
id_29: ['NotFound' 'Found' nan]
id_32: [32. nan 24. 16.  0.]
id_34: ['match_status:2' 'match_status:1' nan 'match_status:0' 'match_status:-1']
id_35: ['T' 'F' nan]
id_36: ['F' nan 'T']
id_37: ['T' 'F' nan]
id_38: ['T' nan 'F']
DeviceType: ['mobile' 'desktop' nan]


__Test transactions dataset:__

In [26]:
test_tran.head(10)

,TransactionID,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,D13,D14,D15,M1,M2,M3,M4,M5,M6,M7,M8,M9,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30,V31,V32,V33,V34,V35,V36,V37,V38,V39,V40,V41,V42,V43,V44,V45,V46,V47,V48,V49,V50,V51,V52,V53,V54,V55,V56,V57,V58,V59,V60,V61,V62,V63,V64,V65,V66,V67,V68,V69,V70,V71,V72,V73,V74,V75,V76,V77,V78,V79,V80,V81,V82,V83,V84,V85,V86,V87,V88,V89,V90,V91,V92,V93,V94,V95,V96,V97,V98,V99,V100,V101,V102,V103,V104,V105,V106,V107,V108,V109,V110,V111,V112,V113,V114,V115,V116,V117,V118,V119,V120,V121,V122,V123,V124,V125,V126,V127,V128,V129,V130,V131,V132,V133,V134,V135,V136,V137,V138,V139,V140,V141,V142,V143,V144,V145,V146,V147,V148,V149,V150,V151,V152,V153,V154,V155,V156,V157,V158,V159,V160,V161,V162,V163,V164,V165,V166,V167,V168,V169,V170,V171,V172,V173,V174,V175,V176,V177,V178,V179,V180,V181,V182,V183,V184,V185,V186,V187,V188,V189,V190,V191,V192,V193,V194,V195,V196,V197,V198,V199,V200,V201,V202,V203,V204,V205,V206,V207,V208,V209,V210,V211,V212,V213,V214,V215,V216,V217,V218,V219,V220,V221,V222,V223,V224,V225,V226,V227,V228,V229,V230,V231,V232,V233,V234,V235,V236,V237,V238,V239,V240,V241,V242,V243,V244,V245,V246,V247,V248,V249,V250,V251,V252,V253,V254,V255,V256,V257,V258,V259,V260,V261,V262,V263,V264,V265,V266,V267,V268,V269,V270,V271,V272,V273,V274,V275,V276,V277,V278,V279,V280,V281,V282,V283,V284,V285,V286,V287,V288,V289,V290,V291,V292,V293,V294,V295,V296,V297,V298,V299,V300,V301,V302,V303,V304,V305,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,V322,V323,V324,V325,V326,V327,V328,V329,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,3663549,18403224,31.95,W,10409,111.0,150.0,visa,226.0,debit,170.0,87.0,1.0,NaN,gmail.com,NaN,6.0,6.0,0.0,0.0,3.0,4.0,0.0,0.0,6.0,0.0,5.0,1.0,115.0,6.0,419.0,419.0,27.0,398.0,27.0,NaN,NaN,NaN,NaN,418.0,203.0,NaN,NaN,NaN,409.0,T,T,F,NaN,NaN,F,T,T,T,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.000000,47.950001,0.000000,0.000000,47.950001,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.000000,47.950001,0.000000,0.000000,47.950001,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3663550,18403263,49.00,W,4272,111.0,150.0,visa,226.0,debit,299.0,87.0,4.0,NaN,aol.com,NaN,3.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,1.0,1.0,12.0,2.0,149.0,149.0,7.0,634.0,7.0,NaN,NaN,NaN,NaN,231.0,634.0,NaN,NaN,NaN,634.0,T,F,F,M0,NaN,F,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0

In [27]:
test_tran.info(memory_usage='deep', verbose=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506691 entries, 0 to 506690
Columns: 393 entries, TransactionID to V339
dtypes: float64(376), int64(3), object(14)
memory usage: 1.8 GB


In [28]:
col_unique_5(test_tran)

ProductCD: ['W' 'C' 'S' 'H' 'R']
card4: ['visa' 'mastercard' 'american express' 'discover' nan]
card6: ['debit' 'credit' nan 'charge card']
M1: ['T' nan 'F']
M2: ['T' 'F' nan]
M3: ['F' 'T' nan]
M4: [nan 'M0' 'M1' 'M2']
M5: [nan 'F' 'T']
M6: ['F' 'T' nan]
M7: ['T' nan 'F']
M8: ['T' nan 'F']
M9: ['T' nan 'F']
V1: [ 1. nan  0.]
V12: [ 0.  1. nan  2.  3.  4.]
V14: [ 1. nan  0.]
V29: [ 0.  1.  2. nan  3.  4.]
V35: [ 1.  2.  0. nan  4.  3.]
V41: [ 1. nan  0.]
V48: [ 1.  0. nan  2.  3.  4.]
V65: [ 1. nan  0.]
V69: [ 0.  1.  2. nan  3.  4.]
V88: [ 1. nan  0.]
V90: [ 1.  0.  2. nan  3.  4.]
V94: [ 0.  1. nan  2.]
V107: [1.]
V117: [1. 2. 0.]
V118: [1. 2. 0. 3.]
V119: [1. 2. 0.]
V120: [1. 2. 0. 3. 4.]
V121: [1. 2. 0. 3. 4.]
V122: [1. 2. 0. 3. 4.]
V240: [nan  1.  2.  6.  3.  5.]
V241: [nan  1.  2.  3.]
V305: [ 1. nan  2.]


__Test identity dataset:__

In [29]:
test_id.head(10)

,TransactionID,id-01,id-02,id-03,id-04,id-05,id-06,id-07,id-08,id-09,id-10,id-11,id-12,id-13,id-14,id-15,id-16,id-17,id-18,id-19,id-20,id-21,id-22,id-23,id-24,id-25,id-26,id-27,id-28,id-29,id-30,id-31,id-32,id-33,id-34,id-35,id-36,id-37,id-38,DeviceType,DeviceInfo
0,3663586,-45.0,280290.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,100.0,NotFound,27.0,NaN,New,NotFound,225.0,15.0,427.0,563.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,New,NotFound,NaN,chrome 67.0 for android,NaN,NaN,NaN,F,F,T,F,mobile,MYA-L13 Build/HUAWEIMYA-L13
1,3663588,0.0,3579.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,100.0,Found,NaN,-300.0,Found,Found,166.0,NaN,542.0,368.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Found,Found,Android 6.0.1,chrome 67.0 for android,24.0,1280x720,match_status:2,T,F,T,T,mobile,LGLS676 Build/MXB48T
2,3663597,-5.0,185210.0,NaN,NaN,1.0,0.0,NaN,NaN,NaN,NaN,100.0,NotFound,52.0,-360.0,New,NotFound,225.0,NaN,271.0,507.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,New,NotFound,NaN,ie 11.0 for tablet,NaN,NaN,NaN,F,T,T,F,desktop,Trident/7.0
3,3663601,-45.0,252944.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,100.0,NotFound,27.0,NaN,Found,Found,225.0,15.0,427.0,563.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Found,Found,NaN,chrome 67.0 for android,NaN,NaN,NaN,F,F,T,F,mobile,MYA-L13 Build/HUAWEIMYA-L13
4,3663602,-95.0,328680.0,NaN,NaN,7.0,-33.0,NaN,NaN,NaN,NaN,100.0,NotFound,27.0,NaN,New,NotFound,225.0,15.0,567.0,507.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,New,NotFound,NaN,chrome 67.0 for android,NaN,NaN,NaN,F,F,T,F,mobile,SM-G9650 Build/R16NW
5,3663622,-5.0,36786.0,NaN,NaN,4.0,-2.0,NaN,NaN,NaN,NaN,100.0,NotFound,27.0,-480.0,New,NotFound,166.0,15.0,352.0,177.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,New,NotFound,iOS 11.4.0,mobile safari 11.0,32.0,2208x1242,match_status:2,T,F,F,T,mobile,iOS Device
6,3663624,-5.0,12577.0,0.0,0.0,2.0,-2.0,NaN,NaN,0.0,0.0,100.0,Found,27.0,-480.0,Found,Found,166.0,15.0,352.0,177.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Found,Found,iOS 11.4.0,mobile safari 11.0,32.0,2208x1242,match_status:2,T,F,F,T,mobile,iOS Device
7,3663626,-5.0,44726.0,0.0,0.0,0.0,-2.0,NaN,NaN,0.0,0.0,100.0,Found,27.0,-480.0,Found,Found,166.0,15.0,352.0,177.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Found,Found,iOS 11.4.0,mobile safari 11.0,32.0,2208x1242,match_status:2,T,F,F,T,mobile,iOS Device
8,3663629,-5.0,125997.0,0.0,0.0,2.0,-2.0,NaN,NaN,0.0,0.0,100.0,Found,27.0,-480.0,Found,Found,166.0,15.0,352.0,177.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Found,Found,iOS 11.4.0,mobile safari 11.0,32.0,2208x1242,match_status:2,T,F,F,T,mobile,iOS Device
9,3663658,-5.0,98741.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,100.0,Found,27.0,-360.0,New,NotFound,166.0,NaN,529.0,214.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,New,NotFound,iOS 11.4.0,mobile safari 11.0,32.0,2048x1536,match_status:2,T,F,F,F,mobile,iOS Device


In [30]:
test_id.info(memory_usage='deep', verbose=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 141907 entries, 0 to 141906
Columns: 41 entries, TransactionID to DeviceInfo
dtypes: float64(23), int64(1), object(17)
memory usage: 154.1 MB


In [31]:
col_unique_5(test_id)

id-12: ['NotFound' 'Found']
id-15: ['New' 'Found' 'Unknown' nan]
id-16: ['NotFound' 'Found' nan]
id-23: [nan 'IP_PROXY:TRANSPARENT' 'IP_PROXY:ANONYMOUS' 'IP_PROXY:HIDDEN']
id-27: [nan 'Found' 'NotFound']
id-28: ['New' 'Found' nan]
id-29: ['NotFound' 'Found' nan]
id-32: [nan 24. 32. 16. 48.  8.]
id-34: [nan 'match_status:2' 'match_status:1']
id-35: ['F' 'T' nan]
id-36: ['F' 'T' nan]
id-37: ['T' 'F' nan]
id-38: ['F' 'T' nan]
DeviceType: ['mobile' 'desktop' nan]


***
Check for duplicates:

In [32]:
# check for duplicates
print(train_tran.duplicated().any())
print(train_id.duplicated().any())
print(test_tran.duplicated().any())
print(test_id.duplicated().any())

False
False
False
False


***
Check for missing values:

In [33]:
# check for missing values
print(train_tran.isna().sum().sum())
print(train_id.isna().sum().sum())
print(test_tran.isna().sum().sum())
print(test_id.isna().sum().sum())

95566686
2104107
73490163
2105385


# Preprocessing

## Merge Datafames

In [34]:
# merge  files
train_df = pd.merge(train_tran, train_id, on='TransactionID', how='left')
test_df = pd.merge(test_tran, test_id, on='TransactionID', how='left')

## Dimensionality reduction on Missing Values

In [35]:
# drop columns containing 80% or more of values missing
old_train = train_df.shape
old_test = test_df.shape
train_df, test_df = drop_na_cols(train_df, test_df)

print(f'Old vs new size of train: {old_train} vs {train_df.shape}')
print(f'Old vs new size of test: {old_test} vs {test_df.shape}')

Old vs new size of train: (590540, 433) vs (590540, 381)
Old vs new size of test: (506691, 433) vs (506691, 381)


## Reduce memory usage

In [36]:
train_df = reduce_mem(train_df)

Original memory usage:

<class 'pandas.core.frame.DataFrame'>
Int64Index: 590540 entries, 0 to 590539
Columns: 381 entries, TransactionID to DeviceInfo
dtypes: float64(347), int64(3), object(31)
memory usage: 2.3 GB

New memory usage:

<class 'pandas.core.frame.DataFrame'>
Int64Index: 590540 entries, 0 to 590539
Columns: 381 entries, TransactionID to DeviceInfo
dtypes: int16(81), int32(53), int8(216), object(31)
memory usage: 1.1 GB


In [37]:
test_df = reduce_mem(test_df)

Original memory usage:

<class 'pandas.core.frame.DataFrame'>
Int64Index: 506691 entries, 0 to 506690
Columns: 381 entries, TransactionID to DeviceInfo
dtypes: float64(347), int64(3), object(31)
memory usage: 2.0 GB

New memory usage:

<class 'pandas.core.frame.DataFrame'>
Int64Index: 506691 entries, 0 to 506690
Columns: 381 entries, TransactionID to DeviceInfo
dtypes: int16(78), int32(53), int8(219), object(31)
memory usage: 947.5 MB


## Save Columns

In [38]:
# keep coplumn names
train_df_cols = train_df.columns.tolist()
test_df_cols = test_df.columns.tolist()
print(train_df_cols == test_df_cols)

False


## Label Encoding

__Categorical Columns:__

Categorical Features - Transaction<br>
`ProductCD`<br>
`card1 - card6`<br>
`addr1, addr2`<br>
`P_emaildomain`<br>
`R_emaildomain`<br>
`M1 - M9`

Categorical Features - Identity<br>
`DeviceType`<br>
`DeviceInfo`<br>
`id_12 - id_38`<br><br>

In [39]:
# label encoding

train_df = lable_enc(train_df)
test_df = lable_enc(test_df)

## Normalizing

In [40]:
# scaling data using Min Max Scaler

train_df = minmax_scale(train_df)
test_df = minmax_scale(test_df)

## Feature Engeneering

In [41]:
# convert the data back to dataframes
train_df = pd.DataFrame(train_df, columns=train_df_cols)
test_df = pd.DataFrame(test_df, columns=test_df_cols)

In [42]:
# aggregate features
train_df = create_features(train_df)
test_df = create_features(test_df)

# Save cleaned data as CSV

In [44]:
cwd = os.getcwd()

# directory for output files
data_dir = os.path.join(cwd, 'data')

# create the directory if it doesn't exist
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

# relative paths
rel_path_cleaned_train = 'cleaned_train.csv'
rel_path_cleaned_test = 'cleaned_test.csv'

# absolute paths
abs_path_cleaned_train = os.path.join(data_dir, rel_path_cleaned_train)
abs_path_cleaned_test = os.path.join(data_dir, rel_path_cleaned_test)

# save the cleaned dataframes to a CSV file
train_df.to_csv(abs_path_cleaned_train, index=False)
test_df.to_csv(abs_path_cleaned_test, index=False)